In [3]:
%pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [1]:
from random import uniform

In [2]:
from pprint import pprint
from pymongo import MongoClient

In [7]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install textblob

In [3]:
import re
import glob
import nltk
nltk.download('punkt') 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

In [5]:
import csv
import pandas as pd

In [6]:
import json
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# TEXT PREPROCESSING And SENTIMENTAL ANALYSIS MODEL

In [7]:
def cleanText(text):
  text = text.lower()
  # Removes all mentions (@username) from the tweet since it is of no use to us
  text = re.sub(r'(@[A-Za-z0-9_]+)', '', text)
    
  # Removes any link in the text
  text = re.sub('http://\S+|https://\S+', '', text)

  # Only considers the part of the string with char between a to z or digits and whitespace characters
  #  removes punctuation
  text = re.sub(r'[^\w\s]', '', text)

  # Removes stop words that have no use in sentiment analysis 
  text_tokens = word_tokenize(text)
  text = [word for word in text_tokens if not word in stopwords.words()]

  text = ' '.join(text)
  return text

In [8]:
def stem(text):
  # This function is used to stem the given sentence
  porter = PorterStemmer()
  token_words = word_tokenize(text)
  stem_sentence = []
  for word in token_words:
    stem_sentence.append(porter.stem(word))
  return " ".join(stem_sentence)

In [9]:
def sentiment(cleaned_text):
  # Returns the sentiment based on the polarity of the input TextBlob object
  if cleaned_text.sentiment.polarity > 0:
    return 'positive'
  elif cleaned_text.sentiment.polarity < 0:
    return 'negative'
  else:
    return 'neutral'

#TWITTER API AND DATA COLLECTION

In [10]:
def Connect():
  client = MongoClient('mongodb://localhost:27017/')
  db = client["twitterdb"]
  return db

In [11]:
def FetchTweet(string,count):
  db = Connect()
  mycol = db[string]

  jsonData = {}

  CONSUMER_KEY = "ThKgOjo33dQWe3vtVZ2u7jYKc"
  CONSUMER_SECRET = "IQZM852eLGSqUfZZLLHq5tlgqhQCKBPWIrDlZ7IOit7L0aXs4g"
  ACCESS_TOKEN = "933127763120099331-8iFAQXEHTm0C80VLOvOyfw0eHEz3yoO"
  ACCESS_TOKEN_SECRET ="fT8s1bSM37KERkcwTsbwpgeF3I5TQX0Z5XlxVdpShixC6"

  auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
  auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
  api = API(auth)

  start_date = "2005-1-1"
  
  tweets = Cursor(api.search_tweets,q=string,lang="en",since=start_date).items(count) # Data in yyyy-mm-dd format

  for tweet in tweets:
  
    txt = tweet.text
    jsonData['tweet'] = txt
    jsonData['name'] = tweet.user.name
    jsonData['created_at'] = str(tweet.created_at)
    jsonData['favorite_count'] = tweet.favourites_count
    jsonData['screen_name'] = str(tweet.user.screen_name)
    jsonData['source'] = tweet.source
    jsonData['retweet_count'] = tweet.retweet_count

    clean_txt = cleanText(txt)
    stem_txt = TextBlob(stem(clean_txt))
    sent = sentiment(stem_txt)
    subj = stem_txt.sentiment.subjectivity
    jsonData['clean tweet'] = clean_txt
    jsonData['Sentiment'] = sent
    jsonData['Subjectivity'] = subj
    mycol.insert_one(jsonData)
    
    print(jsonData)
    jsonData = {}

Unexpected parameter: since


In [ ]:
def FetchLiveTweet(string,count):

    db = Connect()
    mycol = db[string]

    jsonData = {}

    CONSUMER_KEY = "ThKgOjo33dQWe3vtVZ2u7jYKc"
    CONSUMER_SECRET = "IQZM852eLGSqUfZZLLHq5tlgqhQCKBPWIrDlZ7IOit7L0aXs4g"
    ACCESS_TOKEN = "933127763120099331-8iFAQXEHTm0C80VLOvOyfw0eHEz3yoO"
    ACCESS_TOKEN_SECRET ="fT8s1bSM37KERkcwTsbwpgeF3I5TQX0Z5XlxVdpShixC6"

    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    api = API(auth)

    tweets = api.home_timeline(count = count)

    for tweet in tweets:
        txt = tweet.text
        jsonData['tweet'] = txt
        jsonData['name'] = tweet.user.name
        jsonData['created_at'] = str(tweet.created_at)
        jsonData['favorite_count'] = tweet.user.favourites_count
        jsonData['screen_name'] = str(tweet.user.screen_name)
        jsonData['source'] = tweet.source
        jsonData['retweet_count'] = tweet.retweet_count

        clean_txt = cleanText(txt)
        stem_txt = TextBlob(stem(clean_txt))
        sent = sentiment(stem_txt)
        subj = stem_txt.sentiment.subjectivity
        jsonData['clean tweet'] = clean_txt
        jsonData['Sentiment'] = sent
        jsonData['Subjectivity'] = subj
        mycol.insert_one(jsonData)
        
        jsonData = {}

In [85]:
def FetchLiveTweet_company(count):
    companies = ['walmart','Apple','amazon','ford','tesla','nike','google','facebook','Microsoft']
    for company in companies:
        FetchLiveTweet(company,count)

    for company in companies:
        mycol = db[company]
        mycol.update_many({}, [{ "$set": { "created_at": { "$toDate": "$created_at" } }}])

In [12]:
df1 = pd.read_csv('companies.csv')
companies = list(df1['Company']) # generating the list of companies

In [ ]:
FetchTweet('tata',1000)
FetchTweet('benz',1000)
FetchTweet('samsung',1000)
FetchTweet('google',1000)
FetchTweet('nike',1000)

In [13]:
count = 1000 # entering the number of tweets to be fetched

for company in companies:
    try:
        FetchTweet(company,count)

    except:
        pass


NameError: name 'companies' is not defined

In [20]:
#creating a .csv file to be used by dash
col_info1 = ['Name','Positive','Negative','Neutral','Time']
csvfil = open('finval.csv','w')
writ = csv.writer(csvfil)
writ.writerow(col_info1)

37

In [144]:
def aggregate_values(string1):
    db = Connect()
    mycol = db[string1]
    dic = {}
    agg_sentiment = mycol.aggregate([{
        '$group': {
            '_id':{'Sentiment' : '$Sentiment','Year': {'$year':'$created_at'}},
            
            'count':{'$sum':1},
            'sub':{'$sum':'$Subjectivity'}
        }
    }])

    positive = []
    negative = []
    neutral = []
    positive1 = []
    negative1 = []
    neutral1 = []
    ps = []
    ns = []
    nus = []

    for sentiment in agg_sentiment:

        temp = sentiment['_id']


        if(temp['Year'] == 2013):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])


        elif(temp['Year'] == 2014):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])


        elif(temp['Year'] == 2015):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])


        elif(temp['Year'] == 2016):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])
            
            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])
        
        elif(temp['Year'] == 2017):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])
      
        elif(temp['Year'] == 2018):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])


        elif(temp['Year'] == 2019):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

        elif(temp['Year'] == 2020):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

        elif(temp['Year'] == 2021):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])

        elif(temp['Year'] == 2022):

            if temp['Sentiment'] == 'positive':
                dic['positive'] = sentiment['count']
                dic['positive_subjectivity'] = sentiment['sub']
                positive.append(dic['positive'])
                ps.append(dic['positive_subjectivity']/dic['positive'])

            elif temp['Sentiment'] == 'negative':
                dic['negative'] = sentiment['count']
                dic['negative_subjectivity'] = sentiment['sub']
                negative.append(dic['negative'])
                ns.append(dic['negative_subjectivity']/dic['negative'])

            elif temp['Sentiment'] == 'neutral':
                dic['neutral'] = sentiment['count']
                dic['neutral_subjectivity'] = sentiment['sub']
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])
            
            else:
                dic['neutral'] = 0
                dic['neutral_subjectivity'] = 0
                neutral.append(dic['neutral'])
                nus.append(dic['neutral_subjectivity']/dic['neutral'])


    for i,j,z in zip(positive,negative,neutral):
        temp1 = i/(i+j+z)
        positive1.append(temp1)
        temp2 = j/(i+j+z)
        negative1.append(temp2)
        temp3 = z/(i+j+z)
        neutral1.append(temp3)

        
    return positive1,negative1,neutral1,ps,ns,nus 

In [145]:
positive = []
negative = []
neutral = []
ps = []
ns = []
nus = []
positive,negative,neutral,ps,ns,nus = aggregate_values('amazon')
year = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]


In [146]:
dic = {}
dic['year'] = year
dic['positive'] = positive
dic['negative'] = negative
dic['neutral'] = neutral
dic['positive_subjectivity'] = ps
dic['negative_subjectivity'] = ns
dic['neutral_subjectivity'] = nus

df3 = pd.DataFrame(dic)

In [147]:
df3.to_csv('file2.csv')

In [ ]:
db = Connect()
for company in companies:
    mycol = db[company]
    mycol.update_many({}, [{ "$set": { "created_at": { "$toDate": "$created_at" } }}])

    find = mycol.find()
    for i in find:
        pprint(i)

In [16]:
def aggregate(string1):
    db = Connect() # connecting to mongodb
    mycol = db[string1] # collection
    dic = {}
    list1 = []
    list2 = []
    p=0
    n=0
    b =0
    count = 0
    #error = uniform(-10,10)
    for x in mycol.find({},{ "_id": 1, "Sentiment": 1 , "created_at" : 1}): #getting required data from mongodb
        if x['Sentiment'] == 'positive':
            p+=1

        elif x['Sentiment'] == 'negative':
            b+=1

        else:
            n+=1
        count+=1
        list1.append(x['Sentiment'])
        list2.append(x['_id'])

        time = x["created_at"]
    
    apos = round(p/count * 100, 2) 
    aneg = round(b/count * 100, 2) 
    aneu = round(n/count * 100 ,2) 

    dic['Sentiment'] = list1
    dic['_id'] = list2
    df = pd.DataFrame(dic)

    #entering data to csv file to be used by dash
    field = [string1,apos,aneg,aneu,time] 
    csvfil = open('finval.csv','a')
    writ = csv.writer(csvfil)
    writ.writerow(field)
    

In [22]:
for company in companies: # aggregating data from mongoDB for sentiment
    aggregate(company)

ZeroDivisionError: division by zero

In [ ]:
%pip install dash

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install waitress

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#importing packages and libraries
import json
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

#external css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

#inline styling
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

app = Dash(__name__, external_stylesheets=external_stylesheets)
filpath = r'finval.csv'
df = pd.read_csv(filpath)

fig = px.scatter(df, x="Name", y="Negative",hover_name="Name",hover_data=["Positive","Negative"]) #scatter plot

#line plot
fig1 = px.line(df, x="Name", y="Positive")
fig2 = px.line(df, x="Name", y="Neutral")

fig.update_layout(clickmode='event+select')

fig.update_traces(marker_size=20)

app.layout = html.Div([
    html.Div([
        html.H1(children='First Graph'), # scatter plot

        html.Div(children='''
            Graph between Name and Positive values
        '''),

        dcc.Graph(
            id='graph1',
            figure=fig
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H1(children='Second Graph'),

        html.Div(children='''
            Graph between Name and Negative values
        '''),

        dcc.Graph(
            id='graph2',
            figure=fig1
        ),  
    ]),
    dcc.Graph(
        id='basic-interactions',
        figure=fig2
    ),
    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""
                **Hover Data**

                Mouse over values in the graph.
            """),
            html.Pre(id='hover-data', style=styles['pre'])
        ], className='three columns'),
    ])
])


@app.callback(
    Output('hover-data', 'children'),
    Input('basic-interactions', 'hoverData'))


def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)


#deploying the app
if __name__ == '__main__':
    app.run_server()

In [34]:
def aggregate_sum(string1):
    dic = {}
    db = Connect() # connecting to mongodb
    mycol = db[string1]
    agg_sentiment1 = mycol.aggregate([{
        '$group': {
            '_id':{'Sentiment' : '$Sentiment'},
            'count':{'$sum':1}
        }
    }])

    agg_sentiment2 = mycol.aggregate([{
        '$group': {
            '_id':{'Sentiment' : 'Sentiment'},
            'count':{'$sum':1},
            'subjectivity_count':{'$sum':'$Subjectivity'}
        }
    }])


    dic['company'] = string1
    dic['positive'] = 0
    dic['negative'] = 0
    dic['neutral'] = 0
    dic['total_count'] = 0
    dic['total_subjectivity'] =0
    
    for i in agg_sentiment2:
        dic['total_count'] = i['count'] 
        dic['total_subjectivity'] = i['subjectivity_count'] / dic['total_count']

    for j in agg_sentiment1:
    
        temp = j['_id']
        if(temp['Sentiment'] == 'positive'):
            dic['positive'] = j['count']/dic['total_count']

        elif(temp['Sentiment'] == 'negative'):
            dic['negative'] = j['count']/dic['total_count']

        elif(temp['Sentiment'] == 'neutral'):
            dic['neutral'] = j['count']/dic['total_count']

    
    return dic['positive'],dic['negative'],dic['neutral'],dic['total_subjectivity'],dic['total_count']

In [67]:
positive = []
negative = []
neutral = []
total_subjectivity = []
total_count = []
dic = {}
companies = ['walmart','Apple','amazon','ford','tesla','nike','google','facebook','Microsoft']
for company in companies:
    temp1,temp2,temp3,temp4,temp5 = aggregate_sum(company)
    positive.append(temp1)
    negative.append(temp2)
    neutral.append(temp3)
    total_subjectivity.append(temp4)
    total_count.append(temp5)

dic['companies'] = companies
dic['positive'] = positive
dic['negative'] = negative
dic['neutral'] = neutral
dic['total_subjectivity'] = total_subjectivity
dic['total_count'] = total_count

{'company': 'walmart', 'positive': 0.434, 'negative': 0.116, 'neutral': 0.45, 'total_count': 2000, 'total_subjectivity': 0.2580894697200947}
{'company': 'Apple', 'positive': 0.29233333333333333, 'negative': 0.059333333333333335, 'neutral': 0.6483333333333333, 'total_count': 3000, 'total_subjectivity': 0.18517471841430178}
{'company': 'amazon', 'positive': 0.24275534441805224, 'negative': 0.1163895486935867, 'neutral': 0.640855106888361, 'total_count': 2105, 'total_subjectivity': 0.18955438236435862}
{'company': 'ford', 'positive': 0.2786396181384248, 'negative': 0.24045346062052506, 'neutral': 0.4809069212410501, 'total_count': 1676, 'total_subjectivity': 0.32823972003864604}
{'company': 'tesla', 'positive': 0.22857142857142856, 'negative': 0.11527093596059114, 'neutral': 0.6561576354679803, 'total_count': 1015, 'total_subjectivity': 0.22432557038837828}
{'company': 'nike', 'positive': 0.3957654723127036, 'negative': 0.10830618892508144, 'neutral': 0.495928338762215, 'total_count': 122

In [68]:
df1 = pd.DataFrame(dic)

In [69]:
df1

,companies,positive,negative,neutral,total_subjectivity,total_count
0,walmart,0.434000,0.116000,0.450000,0.258089,2000
1,Apple,0.292333,0.059333,0.648333,0.185175,3000
2,amazon,0.242755,0.116390,0.640855,0.189554,2105
3,ford,0.278640,0.240453,0.480907,0.328240,1676
4,tesla,0.228571,0.115271,0.656158,0.224326,1015
5,nike,0.395765,0.108306,0.495928,0.307841,1228
6,google,0.245449,0.093534,0.661017,0.188121,1593
7,facebook,0.335000,0.122000,0.543000,0.219457,1000
8,Microsoft,0.359000,0.129000,0.512000,0.290802,1000


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input,Output

app = dash.Dash(__name__)
app.title = "Brand Sentimental Analysis"

#creating css for main div 
main_div_style = {"background-color": "#181818", 
                    "padding":"0", 
                    "width":"100%", 
                    "height":"100", 
                    "position": "fixed",
                    "top": "0%",
                    "left": "0",
                    "bottom": "0",
                }

#creating dropdown section
dropdown_box = html.Div(children = [
            dcc.Dropdown(
                    id='player_name',              #unique identifier for this element
                    options=companies,               #list of players
                    clearable=False,               #this feature helps in not sending an empty value
                    value = "V Kohli",             #setting a default vali
                    placeholder="Select a player", #not really required but just for fun
             )
            ],
            style = {'width': '90%', "position": "fixed", "left": "5%",
                     'display': 'inline-block', "top": "1%", "z-index":"1"}
)
'''#Creating a section to place our graphs 
Graphs = html.Div(children = [
html.Div(children = ["Graph 1", dcc.Graph()], #dash graph element
style = {'width': '45%', "position": "fixed", "left": "2%",
         'display': 'inline-block', 'text-align': 'center',
         'top': '10%', 'border': '2px solid red', 'height':'50%'}
),
html.Div(children = ["Graph 2", dcc.Graph()], #dash graph element
style = {'width': '45%', 'position': 'fixed', 'left': '53%',
         'display': 'inline-block', 'text-align': 'center',
         'top': '10%', 'border': '3px solid red', 'height':'50%'}
)
])'''


#creating graphs section
Graphs = html.Div(children = [html.Div(children = [
                            dcc.Graph(id = "Positive Sentiment")       #graph only requireds unique id in our case
                                    ],
                            style = {'width': '30%', "position": "fixed", "left": "2%",
                                     'display': 'inline-block', "background-color": "#282828",
                                     "top": "10%",
                                    }
                                    ),

                        html.Div(children = [
                                dcc.Graph(id = "Negative Sentiment") #graph only requireds unique id in our case
                                ],
                        style = {'width': '30%', "position": "fixed", "left": "35%",
                                 'display': 'inline-block', "background-color": "#282828",
                                 "top": "10%"
                                })
                        ]
)
#creating and adding the main html div which encapsulates all elements on the viewport
#adding the layout to the dash app
app.layout = html.Div(id = "main_div", children =[
    html.H1(children='Brand Sentimental Analysis'),
    Graphs
],
                     style = main_div_style )

'''@app.callback([Output(component_id = "runsScored", component_property = "figure"),
               Output(component_id = "wicketsTaken", component_property = "figure")],
               [Input(component_id = "player_name", component_property = "value")]
             )
def the_callback_function(player_name):
    fig1 = get_total_runs(player_name)
    fig2 = get_total_wickets(player_name)
    return fig1, fig2'''

if __name__ == "__main__":
    app.run_server()

In [43]:
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.express as px

In [49]:
df1

,companies,positive,negative,neutral,total_subjectivity,total_count
0,walmart,0.434000,0.116000,0.450000,0.258089,2000
1,Apple,0.292333,0.059333,0.648333,0.185175,3000
2,amazon,0.242755,0.116390,0.640855,0.189554,2105
3,ford,0.278640,0.240453,0.480907,0.328240,1676
4,tesla,0.228571,0.115271,0.656158,0.224326,1015
5,intel,0.270500,0.240000,0.489500,0.244173,2000
6,hp,0.291199,0.104548,0.604253,0.210303,1693
7,oracle,0.269331,0.130773,0.599896,0.219014,1927
8,Pfizer,0.000000,0.000000,0.000000,0.000000,0
9,nike,0.395765,0.108306,0.495928,0.307841,1228


In [81]:
df1.to_csv('file1.csv')

In [197]:
import pandas as pd

companies = ['walmart','Apple','amazon','ford','tesla','nike','google','facebook','Microsoft']
year = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
sentiments = ['positive','negative','neutral']
value = [0.3228346456692913,0.1889763779527559,0.4881889763779528]
app = dash.Dash()
df_bar = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

main_div_style = {"background-color": "#181818"
                }

                
fig = px.bar(df_bar, x="Fruit", y="Amount", color="City", barmode="group")
fig1 = px.bar(df1,x="companies",y="positive",color="total_subjectivity", barmode="group")
fig2 = px.bar(df1,x="companies",y="negative",color="total_subjectivity", barmode="group")
fig3 = px.bar(df1,x="companies",y="neutral",color="total_subjectivity", barmode="group")
fig5 = px.bar(df3, x="year", y="positive", color="positive_subjectivity", barmode="group")
scatter = px.scatter(df1, x="companies", y="total_count",color="companies",size="total_count",symbol="companies",hover_name="companies",hover_data=["positive","negative","neutral","total_subjectivity","total_count"])
histogram = px.histogram(df, x="total_bill")
fig4 = px.bar(df1,x="companies",y="total_subjectivity",color="total_count", barmode="group")
app.layout = html.Div(children=[

    #This is Heading section
    html.Div([
        html.H1(children="Brand Dashboard"),
    ], style={"text-align": "center", "margin": "3rem 0","text-decoration":"underline"}),

   
    #This is Bar chart
    html.Div([
        html.H1(children='Sentiment vs Subjectivity',id="sc-1", style={"text-align": "center", "text-decoration": "underline"}),
        dcc.Graph(
            figure=scatter
        ),
    ],),

    #This is 3 Bar Charts section
    html.Div([
        html.Div([
            html.H1(children='Positive',id="f-1", style={"text-align":"center"}),
            dcc.Graph(
                figure=fig1
            ),
        ], style={"width":"75%"} ),
        html.Div([
            html.H1(children='Negative',id="f-2", style={"text-align":"center"}),
            dcc.Graph(
                figure=fig2
            ),
        ], style={"width": "75%"}),
        html.Div([
            html.H1(children='Netural',id="f-3", style={"text-align":"center"}),
            dcc.Graph(
                figure=fig3
            ),
        ], style={"width": "75%"}),
    ], style={"display": "flex", "justify-content": "space-between"}),

     #This is 1 Histogram at the center
    html.Div([
        html.Div([
            html.H1(children='Subjectivity of tweets',id="f-4", style={"text-align": "center","margin-top":"60px"}),
            dcc.Graph(
                figure=fig4
            ),
        ], style={"width": "35%"}),
    ], style={"display": "flex", "justify-content": "center"}),
    
    #This is 3 dropdown section
    html.Div([
        html.Div([
             dcc.Dropdown(
                    id='com-1',             
                    options=companies,              
                    clearable=False,              
                    value = "Apple",   
                  ),

             ], style={"text-align": "center", "width": "20%"}),

        
        html.Div([
            dcc.Dropdown(
                    id='sent-1',             
                    options=sentiments,              
                    clearable=False,              
                    value = "positive", 
                ),
        ], style={"text-align": "center", "width": "20%"}),

        html.Div([
                 dcc.Dropdown(
                     id='year-1',             
                    options=year,              
                    clearable=False,              
                    value = 2021, 
                    ),

                 ], style={"text-align": "center", "width": "20%"}),

    ], style={"display": "flex", "justify-content": "space-around", "margin-top": "40px"}),



    
    #This is Bar chart and Pie chart section
    html.Div([
        html.Div([
            html.H1(children='year-wise-Sentiment', style={"text-align": "center"}),
            dcc.Graph(
                id="yr-1",
                figure= fig5
            ),
        ], style={"width": "45%"}),

        dcc.Graph(
            id="pie1"
        ),
    ], style={"display": "flex", "justify-content": "space-around", "margin-top":"80px"}),

    html.Div([
        html.Div([
            html.H1(children='Heading 1', style={"text-align":"center"}),
            dcc.Graph(
                id = "ps-1"
            ),
        ], style={"width":"35%"} ),
        html.Div([
            html.H1(children='Heading 2', style={"text-align":"center"}),
            dcc.Graph(
                id ="ns-1"
            ),
        ], style={"width": "35%"}),
        html.Div([
            html.H1(children='Heading 3', style={"text-align":"center"}),
            dcc.Graph(
                id="nu-1"
            ),
        ], style={"width": "35%"}),
    ], style={"display": "flex", "justify-content": "space-between"}),
   
],style={"background-color":"blue"})


@app.callback(
    Output(component_id='yr-1',component_property='figure'),
    Input(component_id='com-1',component_property='value'),
    Input(component_id='sent-1',component_property='value'),
      
)

def update_graph1(company,sentiment1):
    positive = []
    negative = []
    neutral = []
    ps = []
    ns = []
    nus = []
    positive,negative,neutral,ps,ns,nus = aggregate_values(company)
    year = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

    dic = {}
    dic['year'] = year
    dic['positive'] = positive
    dic['negative'] = negative
    dic['neutral'] = neutral
    dic['positive_subjectivity'] = ps
    dic['negative_subjectivity'] = ns
    dic['neutral_subjectivity'] = nus

    df3 = pd.DataFrame(dic)
    string = sentiment1 + "_subjectivity"
    fig5 = px.bar(df3, x="year", y=sentiment1, color=string, barmode="group")
    return fig5

@app.callback(
    Output(component_id='pie1',component_property='figure'),
    Input(component_id='com-1',component_property='value'),
    Input(component_id='year-1',component_property='value'),
)  
    
def update_graph2(company,year1):

    positive = []
    negative = []
    neutral = []
    ps = []
    ns = []
    nus = []
    positive,negative,neutral,ps,ns,nus = aggregate_values(company)
    year = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

    dic = {}
    dic['year'] = year
    dic['positive'] = positive
    dic['negative'] = negative
    dic['neutral'] = neutral
    dic['positive_subjectivity'] = ps
    dic['negative_subjectivity'] = ns
    dic['neutral_subjectivity'] = nus

    sentiments = ['positive','negative','neutral']

    df3 = pd.DataFrame(dic)

    dic = {}
    list1 = []
    df4 = df3[df3['year'] == year1 ]
    temp1 = df4.iloc[0,1]
    temp2 = df4.iloc[0,2]
    temp3 = df4.iloc[0,3]

    list1.append(temp1)
    list1.append(temp2)
    list1.append(temp3)

    dic["values"] = list1
    dic["names"] = sentiments
    df5 = pd.DataFrame(dic)
    fig6 = px.pie(df5,values='values', names='names', title='Distribution of sentiment of company')

    return fig6

@app.callback(
    Output(component_id='ps-1',component_property='figure'),
    Input(component_id='com-1',component_property='value'),
)

def update_graph3(company):
    positive = []
    negative = []
    neutral = []
    ps = []
    ns = []
    nus = []
    positive,negative,neutral,ps,ns,nus = aggregate_values(company)
    year = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

    dic = {}
    dic['year'] = year
    dic['positive'] = positive
    dic['negative'] = negative
    dic['neutral'] = neutral
    dic['positive_subjectivity'] = ps
    dic['negative_subjectivity'] = ns
    dic['neutral_subjectivity'] = nus

    sentiments = ['positive','negative','neutral']

    df3 = pd.DataFrame(dic)

    bar1 = px.bar(df3, x="year", y="positive_subjectivity", color="positive", barmode="group")
    return bar1

@app.callback(
    Output(component_id='ns-1',component_property='figure'),
    Input(component_id='com-1',component_property='value'),
)

def update_graph4(company):
    positive = []
    negative = []
    neutral = []
    ps = []
    ns = []
    nus = []
    positive,negative,neutral,ps,ns,nus = aggregate_values(company)
    year = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

    dic = {}
    dic['year'] = year
    dic['positive'] = positive
    dic['negative'] = negative
    dic['neutral'] = neutral
    dic['positive_subjectivity'] = ps
    dic['negative_subjectivity'] = ns
    dic['neutral_subjectivity'] = nus

    sentiments = ['positive','negative','neutral']

    df3 = pd.DataFrame(dic)

    bar2 = px.bar(df3, x="year", y="negative_subjectivity", color="negative", barmode="group")
    return bar2

@app.callback(
    Output(component_id='nu-1',component_property='figure'),
    Input(component_id='com-1',component_property='value'),
)

def update_graph5(company):
    positive = []
    negative = []
    neutral = []
    ps = []
    ns = []
    nus = []
    positive,negative,neutral,ps,ns,nus = aggregate_values(company)
    year = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

    dic = {}
    dic['year'] = year
    dic['positive'] = positive
    dic['negative'] = negative
    dic['neutral'] = neutral
    dic['positive_subjectivity'] = ps
    dic['negative_subjectivity'] = ns
    dic['neutral_subjectivity'] = nus

    sentiments = ['positive','negative','neutral']

    df3 = pd.DataFrame(dic)

    bar3 = px.bar(df3, x="year", y="neutral_subjectivity", color="neutral", barmode="group")
    return bar3

In [198]:
if __name__ == '__main__':
    #FetchLiveTweet_company(1)
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2022 03:55:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "GET /_favicon.ico?v=2.3.1 HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 03:55:07] "POST /_dash-update-component HTTP/1.1" 